In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt


In [2]:
# Define input shape for images (resize images to 224x224, assuming RGB images)
IMG_SIZE = 224
BATCH_SIZE = 25
EPOCHS_INITIAL = 16
EPOCHS_FINETUNE = 5


In [3]:
# Use ImageDataGenerator for real-time data augmentation to prevent overfitting
train_datagen = ImageDataGenerator(
    rescale=1./255,               # Normalize the images
    rotation_range=20,            # Random rotation
    width_shift_range=0.2,        # Random horizontal shift
    height_shift_range=0.2,       # Random vertical shift
    shear_range=0.2,              # Shear transformation
    zoom_range=0.2,               # Zoom in/out
    horizontal_flip=True,         # Horizontal flip
    fill_mode='nearest'           # Fill mode
)


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for the test set


In [5]:
# Paths to your training and test dataset directories
train_data_path = 'split/train'
test_data_path = 'split/test'


In [6]:
# Loading training data with augmentation
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'   # For multi-class classification
)


Found 14551 images belonging to 10 classes.


In [7]:
# Loading test data
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 3641 images belonging to 10 classes.


In [8]:
# Load a pre-trained EfficientNetB0 model without the top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))


In [9]:
# Freeze the layers of the base model to prevent training
base_model.trainable = False


In [10]:
# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pool the features from the base model
x = Dense(1024, activation='relu')(x)  # Add a dense layer
output = Dense(10, activation='softmax')(x)  # Output layer for 10 classes



In [11]:
# Create the model
model = Model(inputs=base_model.input, outputs=output)







In [12]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [13]:

# Define where to save the best model during training
checkpoint_filepath = 'best_model.h5.keras'


In [14]:
# Add a ModelCheckpoint callback to save the model with the best validation accuracy
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,  # Filepath to save the model
    monitor='val_accuracy',        # Monitor validation accuracy
    save_best_only=True,           # Save only the best model
    mode='max',                    # Maximize the monitored quantity
    verbose=1                      # Print saving information to console
)

In [15]:
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = test_generator.samples // BATCH_SIZE

# Ensure that steps_per_epoch is at least 1
if steps_per_epoch == 0:
    steps_per_epoch = 1

if validation_steps == 0:
    validation_steps = 1

# Train the model with the ModelCheckpoint callback
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS_INITIAL,  # Initial training epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint]  # Add the checkpoint callback here
)



Epoch 1/10


C:\Users\azcos5\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


454/454 ━━━━━━━━━━━━━━━━━━━━ 0s 637ms/step - accuracy: 0.1092 - loss: 2.3095
Epoch 1: val_accuracy improved from -inf to 0.10371, saving model to best_model.h5.keras
454/454 ━━━━━━━━━━━━━━━━━━━━ 353s 747ms/step - accuracy: 0.1092 - loss: 2.3095 - val_accuracy: 0.1037 - val_loss: 2.3011
Epoch 2/10
  1/454 ━━━━━━━━━━━━━━━━━━━━ 2:27 325ms/step - accuracy: 0.0938 - loss: 2.3051

C:\Users\azcos5\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)



Epoch 2: val_accuracy improved from 0.10371 to 0.20000, saving model to best_model.h5.keras
454/454 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0938 - loss: 2.3051 - val_accuracy: 0.2000 - val_loss: 2.2892
Epoch 3/10
454/454 ━━━━━━━━━━━━━━━━━━━━ 0s 638ms/step - accuracy: 0.0994 - loss: 2.3011
Epoch 3: val_accuracy did not improve from 0.20000
454/454 ━━━━━━━━━━━━━━━━━━━━ 337s 737ms/step - accuracy: 0.0993 - loss: 2.3011 - val_accuracy: 0.1043 - val_loss: 2.3013
Epoch 4/10
  1/454 ━━━━━━━━━━━━━━━━━━━━ 2:29 331ms/step - accuracy: 0.1250 - loss: 2.3064
Epoch 4: val_accuracy did not improve from 0.20000
454/454 ━━━━━━━━━━━━━━━━━━━━ 1s 738us/step - accuracy: 0.1250 - loss: 2.3064 - val_accuracy: 0.1200 - val_loss: 2.2865
Epoch 5/10
454/454 ━━━━━━━━━━━━━━━━━━━━ 0s 636ms/step - accuracy: 0.1024 - loss: 2.3007
Epoch 5: val_accuracy did not improve from 0.20000
454/454 ━━━━━━━━━━━━━━━━━━━━ 337s 735ms/step - accuracy: 0.1024 - loss: 2.3007 - val_accuracy: 0.1048 - val_loss: 2.3008
Epoch 6/10

In [16]:
# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
fine_tune_at = len(base_model.layers) // 2  # Unfreeze half of the layers


In [17]:
# Recompile with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [ ]:
# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS_FINETUNE,  # Fine-tuning epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    callbacks=[checkpoint]  # Continue saving the best model
)

Epoch 1/5
454/454 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2692 - loss: 2.1146
Epoch 1: val_accuracy did not improve from 0.20000
454/454 ━━━━━━━━━━━━━━━━━━━━ 1018s 2s/step - accuracy: 0.2695 - loss: 2.1139 - val_accuracy: 0.0650 - val_loss: 2.3467
Epoch 2/5
  1/454 ━━━━━━━━━━━━━━━━━━━━ 15:05 2s/step - accuracy: 0.4062 - loss: 1.5898
Epoch 2: val_accuracy did not improve from 0.20000
454/454 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4062 - loss: 1.5898 - val_accuracy: 0.0400 - val_loss: 2.3660
Epoch 3/5
454/454 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5032 - loss: 1.3410
Epoch 3: val_accuracy improved from 0.20000 to 0.53733, saving model to best_model.h5.keras
454/454 ━━━━━━━━━━━━━━━━━━━━ 970s 2s/step - accuracy: 0.5032 - loss: 1.3409 - val_accuracy: 0.5373 - val_loss: 1.2319
Epoch 4/5
  1/454 ━━━━━━━━━━━━━━━━━━━━ 15:04 2s/step - accuracy: 0.5000 - loss: 1.2152
Epoch 4: val_accuracy did not improve from 0.53733
454/454 ━━━━━━━━━━━━━━━━━━━━ 2s 765us/step - accuracy: 0.50

In [ ]:

# Display validation accuracy for both initial training and fine-tuning
val_acc_initial = history.history['val_accuracy']
val_acc_finetune = fine_tune_history.history['val_accuracy']


In [ ]:
# Final validation accuracy after fine-tuning
final_val_acc = val_acc_finetune[-1]
print(f"Final validation accuracy after fine-tuning: {final_val_acc:.4f}")

In [ ]:

# Visualize validation accuracy over epochs
epochs_initial = range(1, len(val_acc_initial) + 1)
epochs_finetune = range(len(val_acc_initial) + 1, len(val_acc_initial) + len(val_acc_finetune) + 1)

plt.plot(epochs_initial, val_acc_initial, label='Initial Training Val Accuracy')
plt.plot(epochs_finetune, val_acc_finetune, label='Fine-Tuning Val Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
